<h1>Binary Ninja Mermaid Graphs</h1>

In [1]:
%%writefile binary_ninja_cfg.py
import binaryninja

def get_cfg_as_mermaid(filepath, function_address):
    bv = binaryninja.load(filepath)

    def extract_cfg(function):
        cfg = {}
        for block in function.basic_blocks:
            cfg[block.start] = [edge.target.start for edge in block.outgoing_edges]
        return cfg

    def cfg_to_mermaid(cfg):
        mermaid = "```mermaid\ngraph TD\n"
        for src, targets in cfg.items():
            src_hex = f"{src:#x}"
            for tgt in targets:
                tgt_hex = f"{tgt:#x}"
                mermaid += f"    {src_hex} --> {tgt_hex}\n"
        mermaid += "```\n"
        return mermaid

    function = bv.get_function_at(function_address)

    if not function:
        raise ValueError(f"Function not found at address {function_address}")

    cfg = extract_cfg(function)

    mermaid_js = cfg_to_mermaid(cfg)

    return mermaid_js

Overwriting binary_ninja_cfg.py


In [2]:
%%writefile binary_ninja_cfg_func.py
import binaryninja

def get_cfg_as_mermaid(filepath, function_address):
    bv = binaryninja.load(filepath)

    def extract_cfg_and_instructions(function):
        cfg = {}
        instructions = {}
        for block in function.mlil.basic_blocks:
            block_address = block.start
            cfg[block_address] = [edge.target.start for edge in block.outgoing_edges]
            instructions[block_address] = [str(instr) for instr in block]
        return cfg, instructions

    def cfg_to_mermaid(cfg, instructions, function_name, function_start):
        mermaid = f"```mermaid\ngraph TD\n"
        mermaid += f"subgraph {function_name} [{function_name} @ {function_start:#x}]\n"
        for src, targets in cfg.items():
            src_hex = f"{src:#x}"
            src_label = f"{src_hex}\\n" + "\\n".join(instructions[src])
            mermaid += f"    {src_hex}[\"{src_label}\"]\n"
            for tgt in targets:
                tgt_hex = f"{tgt:#x}"
                tgt_label = f"{tgt_hex}\\n" + "\\n".join(instructions[tgt])
                mermaid += f"    {src_hex} --> {tgt_hex}[\"{tgt_label}\"]\n"
        mermaid += "end\n"
        mermaid += "```\n"
        return mermaid

    function = bv.get_function_at(function_address)

    if not function:
        raise ValueError(f"Function not found at address {function_address}")

    cfg, instructions = extract_cfg_and_instructions(function)
    function_name = function.name
    function_start = function.start

    mermaid_js = cfg_to_mermaid(cfg, instructions, function_name, function_start)

    return mermaid_js


Overwriting binary_ninja_cfg_func.py


<h2>Create the Mermaid CFG</h2>

In [3]:
import binary_ninja_cfg

binary_path = "./Tenda/BNDB/trivision_webs.bndb"
function_address = 0xB7E8

mermaid_js = binary_ninja_cfg.get_cfg_as_mermaid(binary_path, function_address)

from IPython.display import Markdown, display
display(Markdown(mermaid_js))

```mermaid
graph TD
    0xb7e8 --> 0xb814
    0xb814 --> 0xb804
    0xb814 --> 0xb81c
    0xb804 --> 0xb820
    0xb804 --> 0xb810
    0xb81c --> 0xb848
    0xb820 --> 0xb848
    0xb820 --> 0xb828
    0xb810 --> 0xb814
    0xb848 --> 0xb850
    0xb828 --> 0xb850
```


In [4]:
import binary_ninja_cfg_func
from IPython.display import Markdown, display

binary_path = "./Tenda/BNDB/trivision_webs.bndb"
function_address = 0xB7E8
mermaid_js = binary_ninja_cfg_func.get_cfg_as_mermaid(binary_path, function_address)
display(Markdown(mermaid_js))

```mermaid
graph TD
subgraph sub_b7e8 [sub_b7e8 @ 0xb7e8]
    0x0["0x0\nr6 = arg1\nr5 = arg2\nresult = 0x9710(arg1)\nresult_1 = result\nr3 = r6 + result\ngoto 6 @ 0xb818"]
    0x0 --> 0x6["0x6\nif (result_1 s>= 0) then 7 @ 0xb804 else 10 @ 0xb81c"]
    0x6["0x6\nif (result_1 s>= 0) then 7 @ 0xb804 else 10 @ 0xb81c"]
    0x6 --> 0x7["0x7\nr2_1 = zx.d([r3].b)\nr3 = r3 - 1\nif (r2_1 == 0x2e) then 11 @ 0xb824 else 12 @ 0xb810"]
    0x6 --> 0xa["0xa\ngoto 14 @ 0xb84c"]
    0x7["0x7\nr2_1 = zx.d([r3].b)\nr3 = r3 - 1\nif (r2_1 == 0x2e) then 11 @ 0xb824 else 12 @ 0xb810"]
    0x7 --> 0xb["0xb\nif (result_1 == 0) then 14 @ 0xb84c else 16 @ 0xb828"]
    0x7 --> 0xc["0xc\nresult_1 = result_1 - 1\ngoto 6 @ 0xb818"]
    0xa["0xa\ngoto 14 @ 0xb84c"]
    0xa --> 0xe["0xe\n[r5].b = 0\ngoto 23 @ 0xb854"]
    0xb["0xb\nif (result_1 == 0) then 14 @ 0xb84c else 16 @ 0xb828"]
    0xb --> 0xe["0xe\n[r5].b = 0\ngoto 23 @ 0xb854"]
    0xb --> 0x10["0x10\nr2_2 = result_1 + 1\nr1 = r6\nr0 = r5\nresult = 0x93e0(r0, r1, r2_2)\nr4 = r5 + result_1\n[r4 + 1].b = 0\ngoto 23 @ 0xb854"]
    0xc["0xc\nresult_1 = result_1 - 1\ngoto 6 @ 0xb818"]
    0xc --> 0x6["0x6\nif (result_1 s>= 0) then 7 @ 0xb804 else 10 @ 0xb81c"]
    0xe["0xe\n[r5].b = 0\ngoto 23 @ 0xb854"]
    0xe --> 0x17["0x17\nreturn result"]
    0x10["0x10\nr2_2 = result_1 + 1\nr1 = r6\nr0 = r5\nresult = 0x93e0(r0, r1, r2_2)\nr4 = r5 + result_1\n[r4 + 1].b = 0\ngoto 23 @ 0xb854"]
    0x10 --> 0x17["0x17\nreturn result"]
    0x17["0x17\nreturn result"]
end
```


In [5]:
from IPython.display import Markdown, display
import binaryninja

bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

function_address = 0xB7E8

func = bv.get_function_at(function_address)

mermaid_syntax = "graph TD;\n"

if func is not None:
    for bb in func.basic_blocks:
        mermaid_syntax += f"BB{hex(bb.start)}(({hex(bb.start)}))\n"

        for frontier in bb.dominators:
            mermaid_syntax += f"BB{hex(bb.start)} --> BB{hex(frontier.start)}\n"
else:
    mermaid_syntax = "No function found at address {hex(function_address)}"

print("```mermaid")
print(mermaid_syntax)
print("```")

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb7e8 --> BB0xb7e8
BB0xb814((0xb814))
BB0xb814 --> BB0xb7e8
BB0xb814 --> BB0xb814
BB0xb804((0xb804))
BB0xb804 --> BB0xb804
BB0xb804 --> BB0xb7e8
BB0xb804 --> BB0xb814
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb81c
BB0xb81c --> BB0xb7e8
BB0xb81c --> BB0xb814
BB0xb820((0xb820))
BB0xb820 --> BB0xb820
BB0xb820 --> BB0xb804
BB0xb820 --> BB0xb7e8
BB0xb820 --> BB0xb814
BB0xb810((0xb810))
BB0xb810 --> BB0xb810
BB0xb810 --> BB0xb804
BB0xb810 --> BB0xb7e8
BB0xb810 --> BB0xb814
BB0xb848((0xb848))
BB0xb848 --> BB0xb848
BB0xb848 --> BB0xb7e8
BB0xb848 --> BB0xb814
BB0xb828((0xb828))
BB0xb828 --> BB0xb820
BB0xb828 --> BB0xb828
BB0xb828 --> BB0xb804
BB0xb828 --> BB0xb7e8
BB0xb828 --> BB0xb814
BB0xb850((0xb850))
BB0xb850 --> BB0xb850
BB0xb850 --> BB0xb7e8
BB0xb850 --> BB0xb814

```


<h2>dominators</h2>

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb7e8 --> BB0xb7e8
BB0xb814((0xb814))
BB0xb814 --> BB0xb7e8
BB0xb814 --> BB0xb814
BB0xb804((0xb804))
BB0xb804 --> BB0xb7e8
BB0xb804 --> BB0xb814
BB0xb804 --> BB0xb804
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb7e8
BB0xb81c --> BB0xb814
BB0xb81c --> BB0xb81c
BB0xb820((0xb820))
BB0xb820 --> BB0xb7e8
BB0xb820 --> BB0xb814
BB0xb820 --> BB0xb804
BB0xb820 --> BB0xb820
BB0xb810((0xb810))
BB0xb810 --> BB0xb7e8
BB0xb810 --> BB0xb814
BB0xb810 --> BB0xb804
BB0xb810 --> BB0xb810
BB0xb848((0xb848))
BB0xb848 --> BB0xb7e8
BB0xb848 --> BB0xb814
BB0xb848 --> BB0xb848
BB0xb828((0xb828))
BB0xb828 --> BB0xb7e8
BB0xb828 --> BB0xb814
BB0xb828 --> BB0xb804
BB0xb828 --> BB0xb820
BB0xb828 --> BB0xb828
BB0xb850((0xb850))
BB0xb850 --> BB0xb7e8
BB0xb850 --> BB0xb814
BB0xb850 --> BB0xb850

```

<h2>post_dominators</h2>

In [6]:
from IPython.display import Markdown, display
import binaryninja

bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

function_address = 0xB7E8

func = bv.get_function_at(function_address)

mermaid_syntax = "graph TD;\n"

if func is not None:
    for bb in func.basic_blocks:
        mermaid_syntax += f"BB{hex(bb.start)}(({hex(bb.start)}))\n"

        for frontier in bb.post_dominators:
            mermaid_syntax += f"BB{hex(bb.start)} --> BB{hex(frontier.start)}\n"
else:
    mermaid_syntax = "No function found at address {hex(function_address)}"

print("```mermaid")
print(mermaid_syntax)
print("```")

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb7e8 --> BB0xb814
BB0xb7e8 --> BB0xb850
BB0xb814((0xb814))
BB0xb814 --> BB0xb850
BB0xb804((0xb804))
BB0xb804 --> BB0xb850
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb848
BB0xb81c --> BB0xb850
BB0xb820((0xb820))
BB0xb820 --> BB0xb850
BB0xb810((0xb810))
BB0xb810 --> BB0xb814
BB0xb810 --> BB0xb850
BB0xb848((0xb848))
BB0xb848 --> BB0xb850
BB0xb828((0xb828))
BB0xb828 --> BB0xb850
BB0xb850((0xb850))

```


```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb7e8 --> BB0xb814
BB0xb7e8 --> BB0xb850
BB0xb814((0xb814))
BB0xb814 --> BB0xb850
BB0xb804((0xb804))
BB0xb804 --> BB0xb850
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb848
BB0xb81c --> BB0xb850
BB0xb820((0xb820))
BB0xb820 --> BB0xb850
BB0xb810((0xb810))
BB0xb810 --> BB0xb814
BB0xb810 --> BB0xb850
BB0xb848((0xb848))
BB0xb848 --> BB0xb850
BB0xb828((0xb828))
BB0xb828 --> BB0xb850
BB0xb850((0xb850))

```

<h2>dominance_frontier</h2>

In [7]:
from IPython.display import Markdown, display
import binaryninja

bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

function_address = 0xB7E8

func = bv.get_function_at(function_address)

mermaid_syntax = "graph TD;\n"

if func is not None:
    for bb in func.basic_blocks:
        mermaid_syntax += f"BB{hex(bb.start)}(({hex(bb.start)}))\n"

        for frontier in bb.dominance_frontier:
            mermaid_syntax += f"BB{hex(bb.start)} --> BB{hex(frontier.start)}\n"
else:
    mermaid_syntax = "No function found at address {hex(function_address)}"

print("```mermaid")
print(mermaid_syntax)
print("```")

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb814
BB0xb804((0xb804))
BB0xb804 --> BB0xb814
BB0xb804 --> BB0xb848
BB0xb804 --> BB0xb850
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb848
BB0xb820((0xb820))
BB0xb820 --> BB0xb848
BB0xb820 --> BB0xb850
BB0xb810((0xb810))
BB0xb810 --> BB0xb814
BB0xb848((0xb848))
BB0xb848 --> BB0xb850
BB0xb828((0xb828))
BB0xb828 --> BB0xb850
BB0xb850((0xb850))

```


```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb814
BB0xb804((0xb804))
BB0xb804 --> BB0xb814
BB0xb804 --> BB0xb848
BB0xb804 --> BB0xb850
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb848
BB0xb820((0xb820))
BB0xb820 --> BB0xb848
BB0xb820 --> BB0xb850
BB0xb810((0xb810))
BB0xb810 --> BB0xb814
BB0xb848((0xb848))
BB0xb848 --> BB0xb850
BB0xb828((0xb828))
BB0xb828 --> BB0xb850
BB0xb850((0xb850))

```

<h2>post_dominance_frontier</h2>

In [8]:
from IPython.display import Markdown, display
import binaryninja

bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

function_address = 0xB7E8

func = bv.get_function_at(function_address)

mermaid_syntax = "graph TD;\n"

if func is not None:
    for bb in func.basic_blocks:
        mermaid_syntax += f"BB{hex(bb.start)}(({hex(bb.start)}))\n"

        for frontier in bb.post_dominance_frontier:
            mermaid_syntax += f"BB{hex(bb.start)} --> BB{hex(frontier.start)}\n"
else:
    mermaid_syntax = "No function found at address {hex(function_address)}"

print("```mermaid")
print(mermaid_syntax)
print("```")

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb804
BB0xb804((0xb804))
BB0xb804 --> BB0xb814
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb814
BB0xb820((0xb820))
BB0xb820 --> BB0xb804
BB0xb810((0xb810))
BB0xb810 --> BB0xb804
BB0xb848((0xb848))
BB0xb848 --> BB0xb814
BB0xb848 --> BB0xb820
BB0xb828((0xb828))
BB0xb828 --> BB0xb820
BB0xb850((0xb850))

```


```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb804
BB0xb804((0xb804))
BB0xb804 --> BB0xb814
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb814
BB0xb820((0xb820))
BB0xb820 --> BB0xb804
BB0xb810((0xb810))
BB0xb810 --> BB0xb804
BB0xb848((0xb848))
BB0xb848 --> BB0xb814
BB0xb848 --> BB0xb820
BB0xb828((0xb828))
BB0xb828 --> BB0xb820
BB0xb850((0xb850))

```

<h2>dominator_tree_children</h2>

In [9]:
from IPython.display import Markdown, display
import binaryninja

bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

function_address = 0xB7E8

func = bv.get_function_at(function_address)

mermaid_syntax = "graph TD;\n"

if func is not None:
    for bb in func.basic_blocks:
        mermaid_syntax += f"BB{hex(bb.start)}(({hex(bb.start)}))\n"

        for frontier in bb.dominator_tree_children:
            mermaid_syntax += f"BB{hex(bb.start)} --> BB{hex(frontier.start)}\n"
else:
    mermaid_syntax = "No function found at address {hex(function_address)}"

print("```mermaid")
print(mermaid_syntax)
print("```")

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb7e8 --> BB0xb814
BB0xb814((0xb814))
BB0xb814 --> BB0xb804
BB0xb814 --> BB0xb81c
BB0xb814 --> BB0xb848
BB0xb814 --> BB0xb850
BB0xb804((0xb804))
BB0xb804 --> BB0xb820
BB0xb804 --> BB0xb810
BB0xb81c((0xb81c))
BB0xb820((0xb820))
BB0xb820 --> BB0xb828
BB0xb810((0xb810))
BB0xb848((0xb848))
BB0xb828((0xb828))
BB0xb850((0xb850))

```


```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb7e8 --> BB0xb814
BB0xb814((0xb814))
BB0xb814 --> BB0xb804
BB0xb814 --> BB0xb81c
BB0xb814 --> BB0xb848
BB0xb814 --> BB0xb850
BB0xb804((0xb804))
BB0xb804 --> BB0xb820
BB0xb804 --> BB0xb810
BB0xb81c((0xb81c))
BB0xb820((0xb820))
BB0xb820 --> BB0xb828
BB0xb810((0xb810))
BB0xb848((0xb848))
BB0xb828((0xb828))
BB0xb850((0xb850))

```

<h2>post_dominator_tree_children</h2>

In [10]:
from IPython.display import Markdown, display
import binaryninja

bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

function_address = 0xB7E8

func = bv.get_function_at(function_address)

mermaid_syntax = "graph TD;\n"

if func is not None:
    for bb in func.basic_blocks:
        mermaid_syntax += f"BB{hex(bb.start)}(({hex(bb.start)}))\n"

        for frontier in bb.post_dominator_tree_children:
            mermaid_syntax += f"BB{hex(bb.start)} --> BB{hex(frontier.start)}\n"
else:
    mermaid_syntax = "No function found at address {hex(function_address)}"

print("```mermaid")
print(mermaid_syntax)
print("```")

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb7e8
BB0xb814 --> BB0xb810
BB0xb804((0xb804))
BB0xb81c((0xb81c))
BB0xb820((0xb820))
BB0xb810((0xb810))
BB0xb848((0xb848))
BB0xb848 --> BB0xb81c
BB0xb828((0xb828))
BB0xb850((0xb850))
BB0xb850 --> BB0xb814
BB0xb850 --> BB0xb804
BB0xb850 --> BB0xb820
BB0xb850 --> BB0xb848
BB0xb850 --> BB0xb828

```


```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb7e8
BB0xb814 --> BB0xb810
BB0xb804((0xb804))
BB0xb81c((0xb81c))
BB0xb820((0xb820))
BB0xb810((0xb810))
BB0xb848((0xb848))
BB0xb848 --> BB0xb81c
BB0xb828((0xb828))
BB0xb850((0xb850))
BB0xb850 --> BB0xb814
BB0xb850 --> BB0xb804
BB0xb850 --> BB0xb820
BB0xb850 --> BB0xb848
BB0xb850 --> BB0xb828

```

<h2>strict_dominators</h2>

In [11]:
from IPython.display import Markdown, display
import binaryninja

bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

function_address = 0xB7E8

func = bv.get_function_at(function_address)

mermaid_syntax = "graph TD;\n"

if func is not None:
    for bb in func.basic_blocks:
        mermaid_syntax += f"BB{hex(bb.start)}(({hex(bb.start)}))\n"

        for frontier in bb.strict_dominators:
            mermaid_syntax += f"BB{hex(bb.start)} --> BB{hex(frontier.start)}\n"
else:
    mermaid_syntax = "No function found at address {hex(function_address)}"

print("```mermaid")
print(mermaid_syntax)
print("```")

```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb7e8
BB0xb804((0xb804))
BB0xb804 --> BB0xb814
BB0xb804 --> BB0xb7e8
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb814
BB0xb81c --> BB0xb7e8
BB0xb820((0xb820))
BB0xb820 --> BB0xb814
BB0xb820 --> BB0xb7e8
BB0xb820 --> BB0xb804
BB0xb810((0xb810))
BB0xb810 --> BB0xb814
BB0xb810 --> BB0xb7e8
BB0xb810 --> BB0xb804
BB0xb848((0xb848))
BB0xb848 --> BB0xb814
BB0xb848 --> BB0xb7e8
BB0xb828((0xb828))
BB0xb828 --> BB0xb814
BB0xb828 --> BB0xb7e8
BB0xb828 --> BB0xb804
BB0xb828 --> BB0xb820
BB0xb850((0xb850))
BB0xb850 --> BB0xb814
BB0xb850 --> BB0xb7e8

```


```mermaid
graph TD;
BB0xb7e8((0xb7e8))
BB0xb814((0xb814))
BB0xb814 --> BB0xb7e8
BB0xb804((0xb804))
BB0xb804 --> BB0xb814
BB0xb804 --> BB0xb7e8
BB0xb81c((0xb81c))
BB0xb81c --> BB0xb814
BB0xb81c --> BB0xb7e8
BB0xb820((0xb820))
BB0xb820 --> BB0xb814
BB0xb820 --> BB0xb804
BB0xb820 --> BB0xb7e8
BB0xb810((0xb810))
BB0xb810 --> BB0xb814
BB0xb810 --> BB0xb804
BB0xb810 --> BB0xb7e8
BB0xb848((0xb848))
BB0xb848 --> BB0xb814
BB0xb848 --> BB0xb7e8
BB0xb828((0xb828))
BB0xb828 --> BB0xb814
BB0xb828 --> BB0xb804
BB0xb828 --> BB0xb7e8
BB0xb828 --> BB0xb820
BB0xb850((0xb850))
BB0xb850 --> BB0xb814
BB0xb850 --> BB0xb7e8

```

<h2>immediate_dominator</h2>

In [12]:
bb = bv.basic_blocks

In [13]:
for bb in func:
    print(bb.immediate_dominator)

None
<BasicBlock: armv7@0xb7e8-0xb804>
<BasicBlock: armv7@0xb814-0xb81c>
<BasicBlock: armv7@0xb814-0xb81c>
<BasicBlock: armv7@0xb804-0xb810>
<BasicBlock: armv7@0xb804-0xb810>
<BasicBlock: armv7@0xb814-0xb81c>
<BasicBlock: armv7@0xb820-0xb828>
<BasicBlock: armv7@0xb814-0xb81c>


<h2>immediate_post_dominator</h2>

In [14]:
for bb in func:
    print(bb.immediate_post_dominator)

<BasicBlock: armv7@0xb814-0xb81c>
<BasicBlock: armv7@0xb850-0xb858>
<BasicBlock: armv7@0xb850-0xb858>
<BasicBlock: armv7@0xb848-0xb850>
<BasicBlock: armv7@0xb850-0xb858>
<BasicBlock: armv7@0xb814-0xb81c>
<BasicBlock: armv7@0xb850-0xb858>
<BasicBlock: armv7@0xb850-0xb858>
None


<h2>References</h2>

- Binary Ninja Python API Reference: https://api.binary.ninja/
- Dominator (graph theory): https://en.wikipedia.org/wiki/Dominator_(graph_theory)
- Compiling Techniques Lecture 14: Building SSA Form: https://opencourse.inf.ed.ac.uk/sites/default/files/2024-02/ct_lecture_14_-_building_ssa_form_0.pdf
- Static Single Assignment 15-411 Compiler Design: https://www.cs.cmu.edu/~fp/courses/15411-f08/lectures/09-ssa.pdf
- GCC Compiler Optimization Course: https://www.youtube.com/watch?v=c6csK4Z_U98&list=PL2saaWTUEfabOcP1xKb64KHNn9vKKPfef